In [1]:
"""
To Make sure the Best Axon method works and also
to see if can decompose the axon any better 
(especially if can help with crosses and parts that aren't skeletonized             )

"""

'\nTo Make sure the Best Axon method works and also\nto see if can decompose the axon any better \n(especially if can help with crosses)\n\n'

In [2]:
%load_ext autoreload
%autoreload 2

In [5]:
from os import sys
sys.path.append("/meshAfterParty/")
sys.path.append("/meshAfterParty/meshAfterParty/")

In [6]:
import datajoint_utils as du
minnie,schema = du.configure_minnie_vm()

INFO - 2021-02-17 17:56:37,242 - settings - Setting database.host to at-database.ad.bcm.edu
INFO - 2021-02-17 17:56:37,244 - settings - Setting database.user to celiib
INFO - 2021-02-17 17:56:37,244 - settings - Setting database.password to newceliipass
INFO - 2021-02-17 17:56:37,249 - settings - Setting stores to {'minnie65': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65', 'stage': '/mnt/dj-stor01/platinum/minnie65'}, 'meshes': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65/02/meshes', 'stage': '/mnt/dj-stor01/platinum/minnie65/02/meshes'}, 'decimated_meshes': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65/02/decimated_meshes', 'stage': '/mnt/dj-stor01/platinum/minnie65/02/decimated_meshes'}, 'skeletons': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65/02/skeletons'}}
INFO - 2021-02-17 17:56:37,250 - settings - Setting enable_python_native_blobs to True
INFO - 2021-02-17 17:56:37,264 - connection - Connect

Connecting celiib@at-database.ad.bcm.edu:3306


INFO - 2021-02-17 17:56:37,580 - settings - Setting enable_python_native_blobs to True
INFO - 2021-02-17 17:56:37,613 - settings - Setting enable_python_native_blobs to True
INFO - 2021-02-17 17:56:37,962 - settings - Setting enable_python_native_blobs to True


In [144]:
import pandas as pd
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import networkx_utils as xu
import datajoint as dj
import neuron_visualizations as nviz
import neuron
import neuron_utils as nru
import proofreading_utils as pru
import classification_utils as clu
import system_utils as su
from importlib import reload

# Checking the Double Axon Neurons

In [117]:
multi_axon_segs,multi_axon_splits = (minnie.AutoProofreadNeurons2() & "cell_type_predicted='excitatory'"
 & "n_axons>1").fetch("segment_id","split_index")

In [118]:
len(multi_axon_segs)

9346

In [119]:
curr_index = 60
segment_id = multi_axon_segs[curr_index]
split_index = multi_axon_splits[curr_index]
segment_id,split_index

(864691134947394812, 0)

In [109]:
du.plot_proofread_neuron(segment_id,
                        split_index=split_index,
                        plot_errored_faces=True,
                        plot_synapses=True,
                        plot_nuclei=True,
                        plot_axon=True)

Synapsse Results:
Total Number of Synapses = 278
n_valid_syn_centers_presyn = 0
n_errored_syn_centers_presyn = 68
n_valid_syn_centers_postsyn = 200
n_errored_syn_centers_postsyn = 10
Including axon mesh
No axon to plot


In [111]:
neuron_obj = du.decomposition_with_spine_recalculation(segment_id,
                                                      split_index=split_index)

Decompressing Neuron in minimal output mode...please wait
Pulled from Table Decomposition so setting split_index = 0
Number of Neurons found = 1


In [112]:
if type(neuron_obj) == tuple:
    test_neuron = neuron_obj[0][split_index]
else:
    test_neuron = neuron_obj

In [113]:
nviz.visualize_neuron(test_neuron,
                     limb_branch_dict="all")


 Working on visualization type: mesh

 Working on visualization type: skeleton


In [107]:
true_axons,true_axon_angles,axon_like_errors = clu.axon_classification(test_neuron,
                        return_axon_angles=True,
                       plot_candidates=False,
                       plot_axons=True,
                        best_axon=True,
                       verbose=True)


Part 1: Axon like branchese 
{'L0': array([ 0,  1,  2,  5,  6,  7,  8,  9, 10, 11, 12, 13, 15, 16, 18, 20, 21,
       25, 28, 30, 33, 34, 35, 36, 37, 39, 40, 41, 55, 56, 57, 58, 59, 60,
       61, 62]), 'L1': array([ 4,  5,  6,  7, 16, 17, 18, 19, 20])}
Limb L0 soma angle: 62.32 
Limb L1 soma angle: 85.85 
Limb L2 soma angle: 92.93 
Limb L3 soma angle: 119.97 

Part 2: possible_axon_limbs = [1, 2, 3]

Phase 2: Working on Limb L1
nodes_to_eliminate = [ 9 11 12]

Phase 2: Working on Limb L2
nodes_to_eliminate = [0]

Phase 2: Working on Limb L3
nodes_to_eliminate = []

----Working on Limb L1-----
No endpoints were eliminated so don't need to add back any candidates

----Working on Limb L2-----
No endpoints were eliminated so don't need to add back any candidates

----Working on Limb L3-----
No nodes were eliminated so don't need to add back any candidates
Final Candidates

Limb 1
Candidate 0 (orange): array([1, 3, 4, 5, 6, 7, 8])
Candidate 1 (crimson): array([10, 13, 14])
Candidate 2 (pa

In [67]:
# """
# Pseudocode: 
# if there even is an axon
# 1) Get the limb name of the best axon
# 2) Get the concept network of the limb and the starting node
# 3) Get the branches that would be axon
# 4) Delete the branches from the concept network
# 5) For each connected component in the leftover network
# - if not have starting node and all in axon-like: 
# add to list to add to the true axon

# 6) add the new nodes to the axon group
# """

# neuron_obj = test_neuron

# if len(best_limb_branch_dict) > 0:
    
#     #1) Get the limb name of the best axon
#     limb_name_of_axon = list(best_limb_branch_dict.keys())
#     if len(limb_name_of_axon) > 1:
#         raise Excpetion("More than 1 axon key")
    
#     limb_name_of_axon = limb_name_of_axon[0]
    
#     curr_limb = neuron_obj[limb_name_of_axon]
#     #2) Get the concept network of the limb and the starting node
#     conc_net = nx.Graph(curr_limb.concept_network)
#     start_node = curr_limb.current_starting_node
    
#     #3) Get the branches that would be axon
#     axon_branches = best_limb_branch_dict[limb_name_of_axon]
#     axon_like_branches = axon_like_limb_branch_dict[limb_name_of_axon]
    
#     #4) Delete the branches from the concept network
#     conc_net.remove_nodes_from(axon_branches)
    
    
#     #5) For each connected component in the leftover network
#     new_axon_branches = []
#     for conn_comp in nx.connected_components(conc_net):
#         conn_comp = list(conn_comp)
        
#         #- if not have starting node and all in axon-like: 
#         #add to list to add to the true axon
#         axon_like_in_conn_comp = np.intersect1d(axon_like_branches,conn_comp)
#         if start_node not in conn_comp and len(axon_like_in_conn_comp) == len(conn_comp):
#             new_axon_branches += conn_comp
    
    
        
#     if len(new_axon_branches) > 0:
#         best_limb_branch_dict[limb_name_of_axon] = np.array(list(axon_branches) + new_axon_branches)
        
# best_limb_branch_dict

{'L6': array([ 8,  9, 11, 13, 18, 20, 21, 22, 23, 24, 25, 26, 27, 15, 17, 12,  0,
         1,  2,  3,  4,  5,  6,  7])}

# Getting a more high fidelity axon skeleton

In [120]:
segment_id = 864691134947394812
split_index = 0

In [121]:
neuron_obj = du.decomposition_with_spine_recalculation(segment_id,
                                                      split_index=split_index)

Decompressing Neuron in minimal output mode...please wait
Pulled from Table Decomposition so setting split_index = 0
Number of Neurons found = 1


In [122]:
nviz.visualize_neuron(neuron_obj,
                     limb_branch_dict="all")


 Working on visualization type: mesh

 Working on visualization type: skeleton


In [127]:
neuron_obj

In [129]:
clu.axon_classification(neuron_obj,
                       best_axon=True)


Part 1: Axon like branchese 
{'L3': array([3, 4]), 'L4': array([1]), 'L6': array([0, 1, 2, 3])}
Limb L0 soma angle: 71.03 
Limb L1 soma angle: 66.27 
Limb L2 soma angle: 78.67 
Limb L3 soma angle: 98.66 
Limb L4 soma angle: 98.86 
Limb L5 soma angle: 100.07 
Limb L6 soma angle: 171.82 

Part 2: possible_axon_limbs = [0, 2, 3, 4, 5, 6]

Phase 2: Working on Limb L0
nodes_to_eliminate = []

Phase 2: Working on Limb L2
nodes_to_eliminate = []

Phase 2: Working on Limb L3
nodes_to_eliminate = [0]

Phase 2: Working on Limb L4
nodes_to_eliminate = [0]

Phase 2: Working on Limb L5
nodes_to_eliminate = []

Phase 2: Working on Limb L6
nodes_to_eliminate = [1]

----Working on Limb L0-----
No nodes were eliminated so don't need to add back any candidates

----Working on Limb L2-----
No nodes were eliminated so don't need to add back any candidates

----Working on Limb L3-----
No endpoints were eliminated so don't need to add back any candidates

----Working on Limb L4-----
No endpoints were elimi

({'L6': array([2, 1, 0, 3])}, {'L3': array([4])})

In [210]:
neuron_obj.axon_mesh, neuron_obj.axon_starting_coordinate

(<trimesh.Trimesh(vertices.shape=(3681, 3), faces.shape=(7325, 3))>,
 array([ 634803., 1033780.,  930241.]))

# getting a higher fidelity axon skeleton

In [208]:
"""



"""

Recieved another instance of Neuron class in init -- so just copying data


In [212]:
[k for k in dir(sk) if "meshparty" in k ]

['meshparty', 'skeletonize_connected_branch_meshparty']

In [220]:
neuron_obj = neuron.Neuron(neuron_obj)

Recieved another instance of Neuron class in init -- so just copying data


In [221]:
neuron_obj.axon_classification(best_axon=True)


Part 1: Axon like branchese 
{'L3': array([3, 4]), 'L4': array([1]), 'L6': array([0, 1, 2, 3])}
Limb L0 soma angle: 71.03 
Limb L1 soma angle: 66.27 
Limb L2 soma angle: 78.67 
Limb L3 soma angle: 98.66 
Limb L4 soma angle: 98.86 
Limb L5 soma angle: 100.07 
Limb L6 soma angle: 171.82 

Part 2: possible_axon_limbs = [0, 2, 3, 4, 5, 6]

Phase 2: Working on Limb L0
nodes_to_eliminate = []

Phase 2: Working on Limb L2
nodes_to_eliminate = []

Phase 2: Working on Limb L3
nodes_to_eliminate = [0]

Phase 2: Working on Limb L4
nodes_to_eliminate = [0]

Phase 2: Working on Limb L5
nodes_to_eliminate = []

Phase 2: Working on Limb L6
nodes_to_eliminate = [1]

----Working on Limb L0-----
No nodes were eliminated so don't need to add back any candidates

----Working on Limb L2-----
No nodes were eliminated so don't need to add back any candidates

----Working on Limb L3-----
No endpoints were eliminated so don't need to add back any candidates

----Working on Limb L4-----
No endpoints were elimi

In [250]:
neuron_obj = neuron.Neuron(neuron_obj)

Recieved another instance of Neuron class in init -- so just copying data


In [388]:
axon_mesh = neuron_obj.axon_mesh
meshparty_sk,sk_meshparty_obj = sk.skeletonize_connected_branch_meshparty(axon_mesh,
                                                        root=neuron_obj.axon_starting_coordinate,
                                                        invalidation_d=1200,
                                                        #combine_close_skeleton_nodes_threshold=20000,
                                                                          filter_end_node_length=3000,
                                                                         )

nviz.plot_objects(axon_mesh,meshparty_sk)

  0%|          | 0/1 [00:00<?, ?it/s]

invalidation_d = 1200
cc_vertex_thresh = 10


100%|██████████| 1/1 [00:00<00:00,  9.26it/s]

connecting at the root
branches_touching_root = [16]
inside filter nodes 
Using Distance measure skeletal_distance



empty_indices % = 0.0
 conflict_indices % = 0.032702537716926834



AFTER face_lookup_resolved_test


In [383]:
debug

> /meshAfterParty/meshAfterParty/meshparty_skeletonize.py(1187)<listcomp>()
   1185             or_idx = np.where(original_br_mapping==j)[0]
   1186             segment_width_measurements_filterd_new.append(np.concatenate([segment_width_measurements_filterd[k] for k in or_idx]))
-> 1187         segment_width_measurements_filterd = segment_width_measurements_filterd_new
   1188 
   1189 

ipdb> u
> /meshAfterParty/meshAfterParty/meshparty_skeletonize.py(1187)skeleton_obj_to_branches()
   1185             or_idx = np.where(original_br_mapping==j)[0]
   1186             segment_width_measurements_filterd_new.append(np.concatenate([segment_width_measurements_filterd[k] for k in or_idx]))
-> 1187         segment_width_measurements_filterd = segment_width_measurements_filterd_new
   1188 
   1189 

ipdb> original_br_mapping
array([ 8,  6,  4,  4,  2,  1,  7,  3,  3,  4,  1,  9,  3,  0, 10,  5,  1])
ipdb> cleaned_branches_faces_filtered
[array([   0,    1,    2, ..., 4310, 4311, 4312]), array

ipdb> len(cleaned_branches_faces_filtered)
11
ipdb> cleaned_branches
[array([[[ 582346.78134098, 1038990.71598894,  968020.65683405],
        [ 582436.64061614, 1038968.90382488,  968058.67425362]],

       [[ 582436.64061614, 1038968.90382488,  968058.67425362],
        [ 582528.62047107, 1038949.99064857,  968092.96404981]],

       [[ 582528.62047107, 1038949.99064857,  968092.96404981],
        [ 582622.25814917, 1038933.68331906,  968123.93108387]],

       ...,

       [[ 620890.39468599, 1043866.78516822,  946264.31728606],
        [ 620966.35098983, 1043904.83152747,  946211.57514767]],

       [[ 620966.35098983, 1043904.83152747,  946211.57514767],
        [ 621042.22831206, 1043940.45869855,  946157.04683711]],

       [[ 621042.22831206, 1043940.45869855,  946157.04683711],
        [ 621096.11187972, 1043965.75899944,  946118.32406955]]]), array([[[ 621556.69048358, 1041614.5594722 ,  946041.47648428],
        [ 621533.27982008, 1041711.15422741,  946052.43253026]],

      

ipdb> len(cleaned_branches)
11
ipdb> len(segment_mesh_faces_filtered)
11
ipdb> segment_width_measurements
[array([472.10608295, 495.9796665 , 535.08792929, 541.04420607,
       534.60325868, 516.30627608, 187.46518827, 114.79486731,
        80.85839592,  75.64463361,  97.04030619, 114.33473646,
        96.52636339, 168.02591869, 195.4148555 ]), array([195.4148555 , 182.76282957, 211.66192033, 110.21187894,
       143.63940699, 120.09742324, 124.88363134, 178.3286068 ,
       105.64926051, 176.16677991, 146.49091058, 543.44356019,
       537.9065442 , 375.32878474, 515.11514423, 446.12360345,
       394.60445222, 345.69509164, 337.02954207, 259.93295907,
       144.5275141 ]), array([144.5275141 , 146.44813788, 117.52467609, 147.14223458,
       150.52125003, 142.10116737, 129.22871189, 208.20277758,
       170.26603024,  57.04453901,  74.24701952, 127.45982828,
       100.50042226, 118.23711093, 151.86532592, 126.44262901,
        79.74828449,  95.59123078, 132.56341525, 116.30410166,


ipdb> len(segment_width_measurements)
17
ipdb> kept_branches_idx
array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10])
ipdb> len(segments)
17
ipdb> len(segment_width_measurements_filterd)
11
ipdb> original_br_mapping
array([ 8,  6,  4,  4,  2,  1,  7,  3,  3,  4,  1,  9,  3,  0, 10,  5,  1])
ipdb> original_br_mapping
array([ 8,  6,  4,  4,  2,  1,  7,  3,  3,  4,  1,  9,  3,  0, 10,  5,  1])
ipdb> len(original_br_mapping)
17
ipdb> c


# Find the biggest border within a radius

In [253]:
neuron_obj = neuron.Neuron(neuron_obj)

Recieved another instance of Neuron class in init -- so just copying data


In [292]:

axon_limb_name = neuron_obj.axon_limb_name
axon_starting_branch = neuron_obj.axon_starting_branch
axon_starting_branch_mesh = neuron_obj[axon_limb_name][axon_starting_branch].mesh

border_vertices_for_axon = tu.largest_border_to_coordinate(
    mesh = axon_starting_branch_mesh,
    coordinate = neuron_obj.axon_starting_coordinate,
    distance_threshold = 10000,
    plot_border_vertices = False,
    error_on_no_border = True,
    plot_winning_border = False,
    verbose = False)

soma_touching_vertices_dict = {0:[border_vertices_for_axon]}

In [351]:
pre = reload(pre)

In [395]:
limb_correspondence_individual,concept_network = pre.preprocess_limb(neuron_obj.axon_mesh,
                   soma_touching_vertices_dict = soma_touching_vertices_dict,
                    meshparty_segment_size = 100,
                    combine_close_skeleton_nodes=True,
                    #combine_close_skeleton_nodes_threshold=1200,
                    combine_close_skeleton_nodes_threshold_meshparty = 1300,
                    filter_end_node_length_meshparty = 3000,
                    filter_end_node_length = 3000,
                                                                     
                    use_meshafterparty=False,
                    perform_cleaning_checks = True,
                    
                    
                    #concept_network parameters
                    run_concept_network_checks = True,
                    return_concept_network = True,
                    return_concept_network_starting_info=False,
                    
                    #printing controls
                    verbose = True,
                    print_fusion_steps=True,
                    
                    check_correspondence_branches = True,
                    filter_end_nodes_from_correspondence=True,
                    error_on_no_starting_coordinates=True,
                    invalidation_d=1200
                                                                     
                    
                   )

  0%|          | 0/1 [00:00<?, ?it/s]

soma_touching_vertices_dict = {0: [TrackedArray([[ 623641.7, 1034075. ,  947366.4],
              [ 623821.9, 1034229. ,  947929.1],
              [ 624175.2, 1034144. ,  947286.8],
              [ 623721. , 1034145. ,  947268. ],
              [ 624044.6, 1033994. ,  947380.2],
              [ 623568.8, 1034218. ,  947619.8],
              [ 624220.8, 1034053. ,  947415.6],
              [ 624245.3, 1034106. ,  948018.2],
              [ 623605.5, 1034229. ,  947719.5],
              [ 624348. , 1034136. ,  947419.4],
              [ 624146.7, 1034260. ,  947994.8],
              [ 624400.8, 1034078. ,  947666.4],
              [ 623810.7, 1034090. ,  947258.9],
              [ 624077.3, 1034144. ,  948076.8],
              [ 624286.3, 1034206. ,  947892. ],
              [ 623668.5, 1034198. ,  947845.5],
              [ 623569.9, 1034176. ,  947412.9],
              [ 624015.8, 1034090. ,  947285.3],
              [ 623947.7, 1034238. ,  947968.7],
              [ 624354.2, 1034118.

100%|██████████| 1/1 [00:00<00:00,  8.65it/s]

meshparty_segment_size = 100
Time for 1st pass MP skeletonization: 0.2435448169708252
connecting at the root
branches_touching_root = [16]
combining close nodes
combine_threshold = 1300
length of Graph = 1701
Working on path [478. 481. 486. 491. 495. 499. 503. 504.]
path_degrees = [3, 2, 2, 2, 2, 2, 2, 3]
After combining close endpoints max(kept_branches_idx) = 16, len(kept_branches_idx) = 16
empty_indices % = 0.0
 conflict_indices % = 0.031432812363572864



AFTER face_lookup_resolved_test



Decomposing first pass: 0.7516913414001465
Only Using MeshParty Skeletonization and Mesh Correspondence
Another print
Divinding into MP and MAP pieces: 2.8133392333984375e-05
Total time for MAP sublimb processing 2.86102294921875e-06
---- Working on MP Decomposition #0 ----
Using Quicker soma_to_piece_touching_vertices because no MAP and only one sublimb_mesh piece 
MP filtering soma verts: 4.363059997558594e-05
Fixing Possible Soma Extension Branch for Sublimb 0
Total time for mesh KDTree = 0.011861085891723633
sbv[0].reshape(-1,3) = [[ 623641.7 1034075.   947366.4]]
closest_sk_pt_coord BEFORE = [ 623818.76410124 1034164.3180282   947391.30119142]
current_skeleton.shape = (142, 2, 3)
node_for_stitch = 73: [ 623818.76410124 1034164.3180282   947391.30119142]
node_for_stitch AFTER = 73: [ 623818.76410124 1034164.3180282   947391.30119142]
possible_node_loc = [11 62 81]
possible_node_loc AFTER = [11 62 81]
curr_shortest_path = [73, 78.0, 81.0]
Current stitch point was not a branch or en


Removed 5 skeletal branches



11466 11467
For marked faces: None
empty_indices % = 0.0
 conflict_indices % = 0.0



AFTER face_lookup_resolved_test


In [396]:
nviz.plot_limb_correspondence(limb_correspondence_individual)

In [246]:
divided_sk = sk.decompose_skeleton_to_branches(meshparty_sk)
sk.find_branch_skeleton_with_specific_coordinate(divided_sk,sk_meshparty_obj.vertices[sk_meshparty_obj.root])

[]

In [247]:
[sk.calculate_skeleton_distance(k) for k in divided_sk]

[47763.95915390842,
 13556.727335851527,
 616.4629750923402,
 3132.462446573434,
 6378.328641826006,
 475.20766853699627,
 26841.13681581898,
 15.825142929781775,
 457.3473286192293,
 462.48825656828444,
 11718.841991358127,
 12102.51949752887,
 3384.3752754630555,
 1512.1687523725527,
 1569.679589167243,
 3687.8296421736527,
 35435.43483677717]

In [249]:
import preprocessing_vp2 as pre

In [ ]:

limb_correspondence_individual,network_starting_info = pre.preprocess_limb(mesh=limb_mesh_mparty,
               soma_touching_vertices_dict = soma_touching_vertices_dict,
               return_concept_network = False, 
               return_concept_network_starting_info=True,
               width_threshold_MAP=500,
               size_threshold_MAP=2000,
               surface_reconstruction_size=1000,  

               #arguments added from the big preprocessing step                                                            
               distance_by_mesh_center=distance_by_mesh_center,
               meshparty_segment_size=meshparty_segment_size,
               meshparty_n_surface_downsampling = meshparty_n_surface_downsampling,

                use_meshafterparty=use_meshafterparty,
                error_on_no_starting_coordinates=True

               )

In [248]:
nviz.plot_objects(axon_mesh,meshparty_sk,
                 scatters=[sk_meshparty_obj.vertices[sk_meshparty_obj.root]],
                 scatters_colors="red",
                 scatter_size=0.03)

In [232]:
sk_meshparty_obj.root

array(0)

array([ 623737.80818985, 1034419.7258438 ,  947591.3691274 ])

In [229]:
dir(sk_meshparty_obj)

['SkeletonIndex',
 '_SkeletonIndex',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_branch_points',
 '_compute_cover_paths',
 '_compute_segments',
 '_cover_paths',
 '_create_branch_and_end_points',
 '_create_default_root',
 '_csgraph_filtered',
 '_edges',
 '_end_points',
 '_kdtree',
 '_node_mask',
 '_pykdtree',
 '_register_skeleton_index',
 '_reset_derived_properties_filtered',
 '_reset_derived_properties_rooted',
 '_rooted',
 '_segment_map',
 '_segments',
 '_single_path_length',
 '_vertices',
 'apply_mask',
 'branch_points',
 'branch_points_undirected',
 'child_nodes',
 'copy',
 'cover_paths',
 'csgraph',
 'csgraph_binary',
 'csgraph_binary_undirected',
 'csgraph_u